Generate State Value Groupings for Chartbook Maps
=======

*Brian Dew*<br>
*September 17, 2017*<br>
*@bd_econ*

In [1]:
# Import preliminaries
import requests
import pandas as pd
import config   ## File with API key
api_key = config.key

# Load list of state names and postal abbreviations
states = pd.read_csv('state_abbrevs.csv').set_index('State')

## Mapping state-level changes to real GDP

Data source: U.S. Bureau of Economic Analysis

https://www.bea.gov/newsreleases/regional/gdp_state/qgsp_newsrelease.htm

In [2]:
# Components of request
base = 'https://www.bea.gov/api/data/?&UserID={}'.format(api_key)
m = '&method=GetData'
dsn = '&datasetname=RegionalProduct'
ind = '&IndustryId=1'
comp = '&Component=RGDP_SQN'
geo = '&GeoFIPS=STATE'
year = '&Year=LAST5'
fmt = '&ResultFormat=json'

# Combined url for request
url = '{}{}{}{}{}{}{}{}'.format(base, m, dsn, ind, comp, geo, year, fmt)

In [3]:
# Make request and load data into pandas dataframe, show last 5 obs
r = requests.get(url).json()
df = pd.DataFrame(r['BEAAPI']['Results']['Data'])
df = df[df['GeoName'].isin(states.index)]
df['date'] = pd.to_datetime(df['TimePeriod'])
df = df.sort_values(['GeoName', 'date'])
df = df.set_index(['GeoName', 'date'])['DataValue']
df = df.apply(pd.to_numeric, errors='coerce').unstack(0)

In [4]:
df.iloc[-1] - df.iloc[8]

GeoName
Alabama                   6368
Alaska                   -1509
Arizona                  19739
Arkansas                  3631
California              191334
Colorado                 21710
Connecticut               2345
Delaware                   725
District of Columbia      5457
Florida                  60033
Georgia                  37217
Hawaii                    3745
Idaho                     4605
Illinois                 25498
Indiana                  16968
Iowa                      5766
Kansas                    4680
Kentucky                  8168
Louisiana                -1855
Maine                     2911
Maryland                 16674
Massachusetts            26188
Michigan                 26231
Minnesota                14250
Mississippi               3638
Missouri                  5602
Montana                    825
Nebraska                  3331
Nevada                   11346
New Hampshire             4918
New Jersey               19363
New Mexico                1330


In [5]:
# Calculate growth rate by state
for s in df.keys():
    df[s] = ((1 + (df[s] - df[s].shift(1))/df[s].shift(1))**4 - 1) * 100

In [6]:
date = '{} Q{}'.format(df.index[-1].year, df.index[-1].quarter)
df1 = df.tail(1).stack().reset_index()[['GeoName', 0]]
df1['Abbrev'] = df1['GeoName'].map(states.to_dict()['Abbreviation'])
df1 = df1.set_index('GeoName').set_value('Michigan', 'Abbrev', 'MI, SP')

gdp_rng = [(-10, -5), (-5, -2), (-2, 0), (0, 2), (2, 5), (5, 10)]

C:\Users\BDew\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [7]:
df1

,0,Abbrev
GeoName,,
Alabama,2.776530,AL
Alaska,3.080210,AK
Arizona,3.059271,AZ
Arkansas,2.488000,AR
California,3.400922,CA
Colorado,3.191187,CO
Connecticut,2.413552,CT
Delaware,0.232235,DE
District of Columbia,1.160540,DC


In [8]:
for i, val in enumerate(gdp_rng):
    print(', '.join(df1[(df1[0] > val[0]) & 
                       (df1[0] <= val[1])]['Abbrev'].values))



NY, ND, SD
DE, DC, IA, MD, MN, MT, NE, NM, OH, WV, WY
AL, AK, AZ, AR, CA, CO, CT, FL, GA, HI, ID, IL, IN, KS, KY, LA, ME, MA, MI, SP, MS, MO, NV, NH, NJ, NC, OK, OR, PA, RI, SC, TN, UT, VT, VA, WA, WI
TX


## Employment-Population Ratio by State

In [4]:
data_url = 'https://www.bls.gov/web/laus/ststdsadata.txt'

cols = ['State', 'Pop', 'LF', 'LFsh', 'Empl', 'EPOP', 'Unempl', 'Unrate']

epop_rng = [(50, 55), (55, 60), (60, 65), (65, 70), (70, 80)]

In [5]:
df = pd.read_table(data_url, header=None).iloc[-54:].reset_index()[0]
month = df[0].strip()
df = df[1:]
df = df.str.strip().str.split('\s\s+', expand=True)
df.columns = cols
df['State'] = [s.split(' .')[0] for s in df['State']]
df = df[(~df['State'].str.contains('city')) & 
        (~df['State'].str.contains('County'))]
df['Abbr'] = df['State'].map(states.to_dict()['Abbreviation'])
df['Abbr'] = df['Abbr'].str.replace('MI', 'MI, SP')
df['EPOP'] = df['EPOP'].astype(float)

In [6]:
for i, val in enumerate(epop_rng):
    print(i)
    print(', '.join(df[(df['EPOP'] > val[0]) & 
                       (df['EPOP'] <= val[1])]['Abbr'].values))

0
AL, MS, NM, WV
1
AZ, AR, CA, DE, FL, KY, LA, MI, SP, MT, NV, NJ, NY, NC, OH, OK, PA, SC, TN
2
AK, CT, GA, HI, ID, IL, IN, KS, ME, MD, MA, MO, OR, RI, TX, VA, WA, WY
3
CO, DC, IA, MN, NE, NH, ND, SD, UT, VT, WI
4



In [7]:
month

'April 2018'

## Unemployment Rate by State

In [8]:
lau_url = 'https://download.bls.gov/pub/time.series/la/'
data_url = '{}la.data.3.AllStatesS'.format(lau_url)
state_url = '{}la.state_region_division'.format(lau_url)

state_codes = pd.read_table(state_url)['srd_code'].to_dict()
s = ['LASST{:02}0000000000003'.format(k) for k, v in state_codes.items()]

urate_rng = [(0,3), (3,4), (4,5), (5,6), (6,7), (7,20)]

In [9]:
df = pd.read_table(data_url, sep='\s+')
month, year = df[df['series_id'] == s[0]][['period', 'year']].iloc[-1]
date = pd.to_datetime('{}-{}-01'.format(year, month[1:]))

In [10]:
data = df[(df['series_id'].isin(s)) & 
          (df['period']==month) & 
          (df['year']==year)].reset_index()
data.loc[:, 'state'] = [state_codes[float(s_id[5:7])] 
                        for s_id in data['series_id']]
data.loc[:, 'abbr'] = data['state'].map(
                        states.to_dict()['Abbreviation'])
data = data.dropna()
data.loc[:, 'abbr'] = data['abbr'].str.replace('MI', 'MI, SP')
data.loc[:, 'value'] = data['value'].astype(float)

In [11]:
for i, val in enumerate(urate_rng):
    print(i)
    d_rng = data[(data['value'] > val[0]) & 
                         (data['value'] <= val[1])]['abbr'].values
    if len(d_rng) > 1:
        print(', '.join(d_rng))
    elif len(d_rng) == 1: 
        print(d_rng.item())
    else:
        print()

0
CO, HI, ID, IA, ME, NE, NH, ND, VT, WI
1
AL, AR, FL, IN, KS, KY, MA, MN, MO, MT, OK, SD, TN, UT, VA, WY
2
AZ, CA, CT, DE, GA, IL, LA, MD, MI, SP, MS, NV, NJ, NY, NC, OH, OR, PA, RI, SC, TX, WA
3
DC, NM, WV
4

5
AK


In [ ]:
data

#### Convert to .tex input



In [ ]:
# Tex strings 
#c_str = '\\tikzset{set state val/.style args={#1}{#1='
#s_str = '\\tikzset{set state val/.list={'

# Color categories
#C1 = '{fill=blue!50!green}}}'
#C2 = '{fill=green!80!blue}}}'
#C3 = '{fill=yellow!70!green}}}'
#C4 = '{fill=orange!60!yellow}}}'
#C5 = '{fill=red!40!orange!80}}}'
#C6 = '{fill=red!80!black!80}}}'

In [ ]:
#S1 = ', '.join(map(str, df1[df1[0] >= 5]['Abbrev'].values))
#S2 = ', '.join(map(str, df1[(df1[0] < 5) & (df1[0] >= 2)]['Abbrev'].values))
#S3 = ', '.join(map(str, df1[(df1[0] < 2) & (df1[0] >= 0)]['Abbrev'].values))
#S4 = ', '.join(map(str, df1[(df1[0] < 0) & (df1[0] >= -2)]['Abbrev'].values))
#S5 = ', '.join(map(str, df1[(df1[0] < -2) & (df1[0] >= -5)]['Abbrev'].values))
#S6 = ', '.join(map(str, df1[df1[0] < -5]['Abbrev'].values))

In [ ]:
#d = {C1: S1, C2: S2, C3: S3, C4: S4, C5: S5, C6: S6}
#for key, value in d.iteritems():
#    print '{}{}\n{}{}}}}}'.format(c_str, key, s_str, value)